In [246]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/67/kwhhh9lj76v79c9zbkp29ms40000gn/T/ipykernel_97055/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [247]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [248]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/Users/tiagovhp/Ironhack/Week_7/Day_1/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

data.head()

(1000, 2)


,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


### Let's divide the training and test set into two partitions

In [249]:
# Your code

# Split into features and labels
features = pd.DataFrame(data['text'],columns=['text'])
labels = pd.DataFrame(data['label'],columns=['label'])

# Lets divide the training data into train and *validation set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=.2)

X_train

,text
385,"General Auditor<br/>First National Bank,<br/>J..."
489,Pls print.
52,Later--how about 10? That way I can leave the ...
114,Turns out the French got through to him today ...
379,PIR <preinesMonday April 26 2010 9:52 PMRe: Ni...
...,...
553,"Hello My Dear Beloved Friend, My name is Mrs. ..."
777,Sorry - first I've heard of it. Philippe and I...
954,"Thank you.MR.Cheung Pui. Managing Director, AD..."
243,Sullivan Jacob J <Sullivann@state.gov>Thursday...


## Data Preprocessing

In [250]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords #list of common words in a language that don't carry much meaning, such as "the", "is", "in", etc.
print(stopwords.words("english")[100:110]) #print a slice of 10 stopwords from the English list

import string
print(string.punctuation) # string of all punctuation characters

from nltk.stem.snowball import SnowballStemmer #stemming algorithm
snowball = SnowballStemmer('english') # creating a SnowballStemmer object for the English language

['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tiagovhp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [251]:
# Your code
import re


cleaned_texts = [] # list to hold cleaned texts
for element in data['text']:
    str_element = str(element)
    # Remove inline CSS:
    cleaned_element = re.sub(r'<style.*?>.*?</style>', '',str_element)
    # Remove HTML comments
    cleaned_element = re.sub(r'<!--.*?-->', '', cleaned_element)
    # Remove remaining HTML tags
    cleaned_element = re.sub(r'<.*?>', '', cleaned_element)
    # Remove special characters (keeping only alphanumeric characters and spaces) -> substitue with space " 
    cleaned_element = re.sub(r'[^a-zA-Z0-9\s]', ' ', cleaned_element)
    # Replace numbers with a space:
    cleaned_element = re.sub(r'\d+', ' ', cleaned_element)
    # Remove single characters from the start
    cleaned_element = re.sub(r'^\s*[a-zA-Z]\s+', '', cleaned_element)
    # Substitute multiple spaces with a single space
    cleaned_element = re.sub(r'\s+', ' ', cleaned_element)
     # Remove the prefixed 'b' if present
    cleaned_element = re.sub(r"^b'|'", '', cleaned_element)
    # Convert to lowercase
    cleaned_element = cleaned_element.lower()

    cleaned_texts.append(cleaned_element)
cleaned_texts=pd.DataFrame(cleaned_texts,columns=['cleaned_text'])
cleaned_texts.head(13)



,cleaned_text
0,dear sir strictly a private business proposal ...
1,will do
2,nora cheryl has emailed dozens of memos about ...
3,dear sir fmadam c i know that this proposal mi...
4,fyi
5,sure bottom line you need a special security c...
6,dear sir i am engr ugo nzego with the engineer...
7,abedin huma saturday november pmhfw quint fmsi...
8,there is an oct th george marshall event at th...
9,for you as the account owner for i and my col...


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

## Now let's work on removing stopwords
Remove the stopwords.

In [252]:
stopwords = stopwords.words("english")
cleaned_texts2=[]
for element in cleaned_texts['cleaned_text']:
    new_element = element
    # Remove stop words from list "element"
    for word in stopwords:
        new_element = re.sub(r'\b' + re.escape(word) + r'\b', '', new_element)
    cleaned_texts2.append(new_element)
cleaned_texts2 = pd.DataFrame(cleaned_texts2,columns=['text'])
cleaned_texts2.head(12)


,text
0,dear sir strictly private business proposal ...
1,
2,nora cheryl emailed dozens memos haiti w...
3,dear sir fmadam c know proposal might sur...
4,fyi
5,sure bottom line need special security code ...
6,dear sir engr ugo nzego engineering stores...
7,abedin huma saturday november pmhfw quint fmsi...
8,oct th george marshall event department ...
9,account owner colleagues set aside ...


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [253]:
# Your code

from nltk.stem import WordNetLemmatizer

# Download WordNet data (only needed once)
nltk.download('wordnet')

# Create a lemmatizer object
lemmatizer = WordNetLemmatizer()

cleaned_texts3 = []
for text in cleaned_texts2['text']:
    # Split the text into words
    words = text.split()
    #Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    # Join the lemmatized words back into a single string
    new_text = ' '.join(lemmatized_words)
    # Append the cleaned text to a new list
    cleaned_texts3.append(new_text)

cleaned_texts3=pd.DataFrame(cleaned_texts3,columns=['text'])

# Display the result
print(cleaned_texts3.head(13))

                                                 text
0   dear sir strictly private business proposal mi...
1                                                    
2   nora cheryl emailed dozen memo haiti weekend p...
3   dear sir fmadam c know proposal might surprise...
4                                                 fyi
5   sure bottom line need special security code ge...
6   dear sir engr ugo nzego engineering store depa...
7   abedin huma saturday november pmhfw quint fmsi...
8   oct th george marshall event department tentat...
9   account owner colleague set aside defray incid...
10  strong http www cnn com world africa kenya cra...
11  dear friend name edward moore qc principal par...
12  compliment today hope forgotten dr mr michaelh...


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tiagovhp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [254]:
# Your code
# Your code
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate the count vector class
v = CountVectorizer()

# Generate the BoW model for our training set
cleaned_texts3_cv = v.fit_transform(cleaned_texts3['text']) #X_train_cv is X_train count vectorizer

# Convert the result to a DataFrame for easier manipulation
cleaned_texts3_cv=pd.DataFrame(cleaned_texts3_cv.toarray(), columns=v.get_feature_names_out())

# Add the labels (spam or ham) to the columns and sum the word counts
cleaned_texts3_cv['label']=data['label']

# Groupby label and sum the word counts
top_words = cleaned_texts3_cv.groupby('label').sum().T
top_words

# Get the top 10 words for ham and spam
top_10_ham = top_words[0].nlargest(10)
top_10_spam = top_words[1].nlargest(10)



# Display results
print("Top 10 Words in Ham Messages:")
print(top_10_ham)

print("\nTop 10 Words in Spam Messages:")
print(top_10_spam)


Top 10 Words in Ham Messages:
state        136
pm           127
would        107
president     99
time          95
call          94
mr            91
obama         84
percent       81
secretary     79
Name: 0, dtype: int64

Top 10 Words in Spam Messages:
money          981
account        895
bank           800
fund           781
transaction    549
business       514
country        508
mr             489
nbsp           475
million        460
Name: 1, dtype: int64


In [255]:
# Your code
# Your code
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate the count vector class
v = CountVectorizer()

# Generate the BoW model for our training set
X_train_cv = v.fit_transform(X_train['text']) #X_train_cv is X_train count vectorizer

# Convert the result to a DataFrame for easier manipulation
X_train_cv=pd.DataFrame(X_train_cv.toarray(), columns=v.get_feature_names_out())

# Add the labels (spam or ham) to the columns and sum the word counts
X_train_cv['label']=y_train['label']

# Groupby label and sum the word counts
top_words = X_train_cv.groupby('label').sum().T
top_words

# Get the top 10 words for ham and spam
top_10_ham = top_words[0].nlargest(10)
top_10_spam = top_words[1].nlargest(10)



# Display results
print("Top 10 Words in Ham Messages:")
print(top_10_ham)

print("\nTop 10 Words in Spam Messages:")
print(top_10_spam)


Top 10 Words in Ham Messages:
the     3456
to      2598
of      2242
and     1935
in      1556
you     1260
this    1062
for      950
that     839
your     820
Name: 0.0, dtype: int64

Top 10 Words in Spam Messages:
the     2564
to      1986
of      1736
and     1379
you     1169
in      1116
this     963
for      766
my       692
your     679
Name: 1.0, dtype: int64


## Extra features

In [256]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

NameError: name 'data_train' is not defined

## How would work the Bag of Words with Count Vectorizer concept?

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code